<a href="https://colab.research.google.com/github/Sallb679/UMD-Professor-Ratings-Analysis/blob/main/HW4_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
reviews = pd.read_csv('/content/drive/MyDrive/HW4(Planet Terp ML analysis)/reviews.csv')
professors = pd.read_csv('/content/drive/MyDrive/HW4(Planet Terp ML analysis)/professors.csv')
grades = pd.read_csv('/content/drive/MyDrive/HW4(Planet Terp ML analysis)/grades.csv')

In [ ]:
reviews.head()

,professor,course,review,rating,expected_grade,created
0,Abani Pradhan,NaN,Pretty funny guy and he knows his stuff too. P...,5,B+,2022-12-11T04:18:01.840304Z
1,Abani Pradhan,NFSC112,Easy class as long as you actually watch the l...,2,B-,2023-12-06T04:55:26.042809Z
2,Abani Pradhan,NFSC112,"Didn't learn anything, but didn't need to. Eas...",4,A,2023-12-23T18:57:10.860075Z
3,Abhijit Dasgupta,ENAE202,I took the ENAE version of this class but it i...,5,A-,2019-12-16T17:37:20Z
4,Abhijit Dasgupta,ENME473,"The man. Really excellent lecturer, he puts in...",5,A,2022-01-21T06:06:28Z


In [ ]:
grades.head()

,course,professor,semester,section,A+,A,A-,B+,B,B-,C+,C,C-,D+,D,D-,F,W,Other
0,NFSC112,Abani Pradhan,201408,0102,2,13,7,5,2,0,0,0,0,0,0,0,0,4,0
1,NFSC112,Abani Pradhan,201408,0105,2,12,8,5,0,1,0,0,0,0,0,0,0,3,0
2,NFSC112,Abani Pradhan,201508,0105,0,10,12,2,4,0,1,0,0,0,0,0,0,1,0
3,NFSC112,Abani Pradhan,201508,0106,2,7,5,5,3,2,1,0,0,0,0,0,2,2,0
4,NFSC112,Abani Pradhan,201508,FC01,1,7,9,9,3,0,0,0,0,0,0,0,0,1,0


In [ ]:
professors.head()

,courses,average_rating,type,name,slug
0,"['INST737', 'ENPM808W', 'ENPM808W', 'ENPM808W'...",NaN,professor,A Seyed,seyed
1,"['MUSC800W', 'MUSC830W', 'MUSC830W', 'MUSC830W']",NaN,professor,Aaron Goldman,goldman_aaron
2,"['THET678', 'THET499']",NaN,professor,Aaron Posner,posner
3,"['NFSC431', 'NFSC679R', 'NFSC112', 'HLTH672', ...",3.6667,professor,Abani Pradhan,pradhan
4,"['CMLT235', 'ENGL101', 'ENGL234', 'ENGL101H', ...",NaN,professor,Abbey Morgan,morgan_abbey


In [ ]:
# 1. Copy
df = reviews.copy()

In [ ]:
from dateutil.parser import isoparse

# Apply the ISO parser to every string
df['created'] = df['created'].apply(isoparse)

# (Optional) ensure UTC timezone
df['created'] = df['created'].dt.tz_convert('UTC')

In [ ]:
df['course']    = df['course'].str.strip()
df['professor'] = df['professor'].str.strip()


In [ ]:
df.head()

,professor,course,review,rating,expected_grade,created
0,Abani Pradhan,NaN,Pretty funny guy and he knows his stuff too. P...,5,B+,2022-12-11 04:18:01.840304+00:00
1,Abani Pradhan,NFSC112,Easy class as long as you actually watch the l...,2,B-,2023-12-06 04:55:26.042809+00:00
2,Abani Pradhan,NFSC112,"Didn't learn anything, but didn't need to. Eas...",4,A,2023-12-23 18:57:10.860075+00:00
3,Abhijit Dasgupta,ENAE202,I took the ENAE version of this class but it i...,5,A-,2019-12-16 17:37:20+00:00
4,Abhijit Dasgupta,ENME473,"The man. Really excellent lecturer, he puts in...",5,A,2022-01-21 06:06:28+00:00


In [ ]:
len(df)

37401

In [ ]:
df = df.drop_duplicates(subset=['professor','course','review','created'])


In [ ]:
len(df)

37365

filling missing course info with unknown because review is still useful

In [ ]:
df['course'] = df['course'].fillna('unknown')

<ipython-input-14-d1d27cfef446>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['course'] = df['course'].fillna('unknown')


In [ ]:
import re
#cleaning function
def clean_text_keep_info(text):
    text = str(text).lower()
    # keep letters, digits, whitespace, apostrophes, ! and ?
    text = re.sub(r"[^a-z0-9\s'!?]", ' ', text)
    # collapse runs of spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [ ]:
# Remove rows where the original review is NaN
df = df.dropna(subset=['review'])

# Re-run your cleaning on review_clean if you’ve already created it:
df['review_clean'] = df['review'].apply(clean_text_keep_info)


In [ ]:
# 1. Map expected_grade → numeric
grade_map = {
    'A+': 4.3, 'A': 4.0, 'A-': 3.7,
    'B+': 3.3, 'B': 3.0, 'B-': 2.7,
    'C+': 2.3, 'C': 2.0, 'C-': 1.7,
    'D+': 1.3, 'D': 1.0, 'D-': 0.7,
    'F':  0.0
}
df['exp_grade_num'] = (
    df['expected_grade']
      .astype(str)
      .str.upper()
      .str.strip()
      .map(grade_map)
)

# 2. Compute review text length features
df['review_word_count'] = df['review_clean'].str.split().str.len()
df['review_char_count'] = df['review_clean'].str.len()

# 3. Quick sanity check
df[['professor','exp_grade_num','review_word_count','review_char_count']].head()


,professor,exp_grade_num,review_word_count,review_char_count
0,Abani Pradhan,3.3,28,135
1,Abani Pradhan,2.7,79,424
2,Abani Pradhan,4.0,56,297
3,Abhijit Dasgupta,3.7,78,415
4,Abhijit Dasgupta,4.0,77,388


In [ ]:
len(df)

37360

Onto the grades table

In [ ]:
# Copy the raw grades table
g = grades.copy()

# Ensure semester & section are strings
g['semester'] = g['semester'].astype(str)
g['section']  = g['section'].astype(str)

# Define all the letter-grade columns
letter_cols = ['A+','A','A-','B+','B','B-',
               'C+','C','C-','D+','D','D-',
               'F','W','Other']

# Fill any missing counts with 0 and convert to int
g[letter_cols] = g[letter_cols].fillna(0).astype(int)

# Quick check
print(g.dtypes[['semester','section'] + letter_cols])


semester    object
section     object
A+           int64
A            int64
A-           int64
B+           int64
B            int64
B-           int64
C+           int64
C            int64
C-           int64
D+           int64
D            int64
D-           int64
F            int64
W            int64
Other        int64
dtype: object


In [ ]:
g['course']    = g['course'].str.strip()
g['professor'] = g['professor'].str.strip()


creating numeric gpa calculation for training in future

In [ ]:
# Sum across all letter-grade columns
g['total_enrolled'] = g[letter_cols].sum(axis=1)


# Compute average GPA

# 4.1 UMD grade-point mapping (A+ and A both = 4.0)
umd_points = {
    'A+': 4.0, 'A': 4.0, 'A-': 3.7,
    'B+': 3.3, 'B': 3.0, 'B-': 2.7,
    'C+': 2.3, 'C': 2.0, 'C-': 1.7,
    'D+': 1.3, 'D': 1.0, 'D-': 0.7,
    'F':  0.0
}

# 4.2 Only include graded columns in the calculation
graded_cols = [c for c in letter_cols if c in umd_points]

# 4.3 Numerator: sum(count * quality points)
g['gpa_sum'] = sum(g[col] * umd_points[col] for col in graded_cols)

# 4.4 Denominator: number of students whose grades count
g['graded_count'] = g[graded_cols].sum(axis=1)

# 4.5 Final UMD average GPA
g['avg_gpa'] = g['gpa_sum'] / g['graded_count']


# --- Quick check of the new features ---
g[['course','professor','semester','section',
   'total_enrolled','graded_count','avg_gpa']].head()




,course,professor,semester,section,total_enrolled,graded_count,avg_gpa
0,NFSC112,Abani Pradhan,201408,0102,33,29,3.737931
1,NFSC112,Abani Pradhan,201408,0105,31,28,3.742857
2,NFSC112,Abani Pradhan,201508,0105,30,29,3.631034
3,NFSC112,Abani Pradhan,201508,0106,29,27,3.248148
4,NFSC112,Abani Pradhan,201508,FC01,30,29,3.586207


In [ ]:
# --- Ensure g has a year column ---
g['year'] = g['semester'].str[:4].astype(int)

g.head()

,course,professor,semester,section,A+,A,A-,B+,B,B-,...,D,D-,F,W,Other,total_enrolled,gpa_sum,graded_count,avg_gpa,year
0,NFSC112,Abani Pradhan,201408,0102,2,13,7,5,2,0,...,0,0,0,4,0,33,108.4,29,3.737931,2014
1,NFSC112,Abani Pradhan,201408,0105,2,12,8,5,0,1,...,0,0,0,3,0,31,104.8,28,3.742857,2014
2,NFSC112,Abani Pradhan,201508,0105,0,10,12,2,4,0,...,0,0,0,1,0,30,105.3,29,3.631034,2015
3,NFSC112,Abani Pradhan,201508,0106,2,7,5,5,3,2,...,0,0,2,2,0,29,87.7,27,3.248148,2015
4,NFSC112,Abani Pradhan,201508,FC01,1,7,9,9,3,0,...,0,0,0,1,0,30,104.0,29,3.586207,2015


In [ ]:
len(g)

71543

In [ ]:
# 6.1 Drop exact duplicate rows if they exist
g = g.drop_duplicates(subset=['course','professor','semester','section'])
len(g)

71543

Now the professors table cleaning

In [ ]:
# 1.1 Copy the raw professors table
p = professors.copy()

# 1.2 Strip extra whitespace and standardize casing
p['name'] = p['name'].str.strip()
p['slug'] = p['slug'].str.lower().str.strip()

p.head()

,courses,average_rating,type,name,slug
0,"['INST737', 'ENPM808W', 'ENPM808W', 'ENPM808W'...",NaN,professor,A Seyed,seyed
1,"['MUSC800W', 'MUSC830W', 'MUSC830W', 'MUSC830W']",NaN,professor,Aaron Goldman,goldman_aaron
2,"['THET678', 'THET499']",NaN,professor,Aaron Posner,posner
3,"['NFSC431', 'NFSC679R', 'NFSC112', 'HLTH672', ...",3.6667,professor,Abani Pradhan,pradhan
4,"['CMLT235', 'ENGL101', 'ENGL234', 'ENGL101H', ...",NaN,professor,Abbey Morgan,morgan_abbey


In [ ]:
p['average_rating'].value_counts(dropna=False)

,count
average_rating,
NaN,8885
5.0000,1559
4.0000,352
3.0000,257
1.0000,184
...,...
4.7419,1
2.5862,1
4.6552,1


In [ ]:
# 1. Original full set of professors
orig_profs = set(professors['name'])

# 2. Professors you’re keeping (from reviews or grades)
rev_profs   = set(df['professor'])
grade_profs = set(g['professor'])
keep_profs  = rev_profs | grade_profs

# 3. Compute the drop‐list
drop_profs = orig_profs - keep_profs

# 4. Inspect
print(f"Total professors originally: {len(orig_profs)}")
print(f"Professors with data to keep: {len(keep_profs)}")
print(f"Professors to drop: {len(drop_profs)}")
print("Sample to drop:", list(drop_profs)[:10])



Total professors originally: 13421
Professors with data to keep: 4536
Professors to drop: 8885
Sample to drop: ['Sherman Robinson', 'Fereshteh Shahoveisi', 'Liangjun Shi', 'Emily Summers', 'Alexander Pyles', 'Feng Zhao', 'Nicholas Pagnucco', 'Nancy Truong', 'Michael Quinlivan', 'Oluchi-Haimbach Ofoha']


In [ ]:
p = p[p['name'].isin(keep_profs)]


In [ ]:
import ast

# 1. Parse the string‐repr into real lists
def parse_courses(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return []

p['courses'] = p['courses'].apply(parse_courses)

# 2. Recompute num_courses
p['num_courses'] = p['courses'].apply(len)

# 3. Quick check
p[['name','num_courses']].head(10)

,name,num_courses
3,Abani Pradhan,14
6,Abhijit Dasgupta,23
8,Abigail McEwen,26
11,Abolhassan Jawahery,9
12,Abram Kagan,9
14,Adam Binkley,6
15,Adam Hsieh,12
17,Adam Porter,6
18,Adi Mahalel,45
19,Adil Hassam,11


In [ ]:
# 1. Aggregate teaching history from the grades table
teach_agg = (
    g
    .groupby('professor')
    .agg(
        num_semesters_taught=('semester','nunique'),
        num_years_taught    =('year','nunique')
    )
)

# 2. Merge into your professors table
#    (assuming p['name'] matches g['professor'])
p = p.merge(
    teach_agg,
    left_on='name',
    right_index=True,
    how='left'
)

# 3. Fill any professors who somehow have no sections with zero
p['num_semesters_taught'] = p['num_semesters_taught'].fillna(0).astype(int)
p['num_years_taught']     = p['num_years_taught'].fillna(0).astype(int)

# 4. Inspect
p[['name','num_courses','num_semesters_taught','num_years_taught']].head()


,name,num_courses,num_semesters_taught,num_years_taught
3,Abani Pradhan,14,7,7
6,Abhijit Dasgupta,23,4,4
8,Abigail McEwen,26,8,7
11,Abolhassan Jawahery,9,11,10
12,Abram Kagan,9,6,5


In [ ]:
# Count how many reviews each professor has
rev_count = (
    df
    .groupby('professor')
    .size()
    .rename('review_count')
)
# Bring review_count into p (matching p['name'] to rev_count’s index)
p = (
    p
    .merge(rev_count, left_on='name', right_index=True, how='left')
)

# Professors with zero reviews get NaN → fill with 0
p['review_count'] = p['review_count'].fillna(0).astype(int)

p[['name','review_count']].sort_values('review_count', ascending=False).head()


,name,review_count
8723,Magdalene Ngeve,441
1626,Larry Herman,302
1410,Justin Wyss-Gallifent,297
872,Fawzi Emad,292
2110,Nelson Padua-Perez,235


Combining all the tables

In [ ]:
print(df.columns.tolist())
print(df.dtypes)

['professor', 'course', 'review', 'rating', 'expected_grade', 'created', 'review_clean', 'exp_grade_num', 'review_word_count', 'review_char_count']
professor                         object
course                            object
review                            object
rating                             int64
expected_grade                    object
created              datetime64[ns, UTC]
review_clean                      object
exp_grade_num                    float64
review_word_count                  int64
review_char_count                  int64
dtype: object


In [ ]:
# 1. Review-level aggregates → one row per professor
rev_feats = (
    df
    .groupby('professor')
    .agg(
        review_count            = ('review_clean',     'size'),
        avg_expected_grade      = ('exp_grade_num',    'mean'),
        avg_review_word_count   = ('review_word_count','mean'),
        avg_review_char_count   = ('review_char_count','mean')
    )
    .reset_index()
    .rename(columns={'professor':'name'})
)

# 2. Grade-level aggregates → one row per professor
grade_feats = (
    g
    .groupby('professor')
    .agg(
        total_enrolled_prof = ('total_enrolled','sum'),
        graded_count_prof   = ('graded_count',   'sum'),
        avg_gpa_prof        = ('avg_gpa',        'mean')
    )
    .reset_index()
    .rename(columns={'professor':'name'})
)

# 3. Merge onto the professor table
prof_feats = ['name','average_rating','num_courses',
              'num_semesters_taught','num_years_taught']

main = (
    p[prof_feats]
    .merge(rev_feats,   on='name', how='left')
    .merge(grade_feats, on='name', how='left')
)

# 4. Fill missing numeric features with 0
for col in ['review_count','avg_expected_grade','avg_review_word_count',
            'avg_review_char_count','total_enrolled_prof',
            'graded_count_prof','avg_gpa_prof']:
    main[col] = main[col].fillna(0)

# Save to your Drive folder
main.to_csv('/content/drive/MyDrive/HW4(Planet Terp ML analysis)/main.csv', index=False)


In [ ]:
main.shape

(4538, 12)

In [ ]:
main.head()


,name,average_rating,num_courses,num_semesters_taught,num_years_taught,review_count,avg_expected_grade,avg_review_word_count,avg_review_char_count,total_enrolled_prof,graded_count_prof,avg_gpa_prof
0,Abani Pradhan,3.6667,14,7,7,3,3.333333,54.333333,285.333333,528.0,490.0,3.550642
1,Abhijit Dasgupta,4.2500,23,4,4,4,3.675000,142.250000,769.250000,190.0,179.0,3.478454
2,Abigail McEwen,2.6364,26,8,7,11,4.000000,131.909091,705.181818,640.0,615.0,3.710212
3,Abolhassan Jawahery,4.4000,9,11,10,5,3.200000,72.400000,404.200000,255.0,235.0,3.314467
4,Abram Kagan,2.7000,9,6,5,10,3.470000,162.600000,923.400000,151.0,136.0,3.087617
